In [1]:
import numpy  as np
import math
import os


In [99]:
def read_final(filename):
    with open (filename) as f:
        lines = f.readlines()
    atoms = np.array([0,0,0,0,0])
    for i in range(9,len(lines)):
        atoms = np.vstack((atoms,np.array(lines[i].split(),dtype = float)))
    atoms = np.delete(atoms,0,axis = 0)
    atoms = np.delete(atoms,[0,1],1)
    a = float(lines[5].split()[1])
    b = float(lines[6].split()[1])
    c = float(lines[7].split()[1])
    return atoms

In [107]:
def read_atominfile(filename):
    #read atom positions from POSCAR
    with open (f"{filename}",'r') as f:
        lines = f.readlines()
    atoms = np.array([0,0,0,0,0])
    for i in range(14,len(lines)):
        atoms = np.vstack((atoms,np.array(lines[i].split(),dtype = float)))
    atoms = np.delete(atoms,0,axis= 0)
    atoms = np.delete(atoms, [0], 1)
    a = float(lines[6].split()[1])
    b = float(lines[7].split()[1])
    c = float(lines[8].split()[1])
    return a, b, c, atoms

In [64]:
a, b, c, atoms_nocenter = read_atomin_id("530_nocenter")

In [66]:
a, b, c, atoms_center = read_atomin_id("530_center")

In [68]:
tol = 1e-5

In [69]:
center = 105.54022915

In [76]:
atoms_c = atoms_center[(atoms_center[:,1]>center-tol) & (atoms_center[:,1]<center+tol)]

In [77]:
atoms_need_modify = atoms_center[(atoms_center[:,1]<center-tol) | (atoms_center[:,1]>center+tol)]

In [79]:
atoms_need_modify[:,1] = center

In [80]:
reduced_atoms = np.unique(atoms_need_modify,axis=0)

In [82]:
len(reduced_atoms)

12

In [83]:
atoms_newcenter = np.vstack((atoms_c,reduced_atoms))

In [85]:
atoms_final = np.vstack((atoms_nocenter,atoms_newcenter))

In [63]:
def read_atomin_id(filename):
    #read atom positions from POSCAR
    with open (f"{filename}",'r') as f:
        lines = f.readlines()
    atoms = np.array([0,0,0,0,0])
    for i in range(13,len(lines)):
        atoms = np.vstack((atoms,np.array(lines[i].split(),dtype = float)))
    atoms = np.delete(atoms,0,axis= 0)
    atoms = np.delete(atoms,[0],1)
    a = float(lines[5].split()[1])
    b = float(lines[6].split()[1])
    c = float(lines[7].split()[1])
    return a, b, c, atoms

In [160]:
def Write_to_lammps(a,b,c,supercell_atoms,filename):
    dim = np.array([1,1,1])
    X = supercell_atoms.copy()

    NumberAt = len(X) 

    dimx, dimy, dimz = dim

    xlo = 0.00000000
    xhi = a
    ylo = 0.00000000
    yhi = b 
    zlo = 0.00000000
    zhi = c

    yz = 0.0

    Counter = np.arange(1, NumberAt + 1).reshape(1, -1)

    # data = np.concatenate((X_new, Y_new))
    FinalMat = np.concatenate((Counter.T, X), axis=1)

    with open(filename, 'w') as f:
        f.write('#Header \n \n')
        f.write('{} atoms \n \n'.format(NumberAt))
        f.write('2 atom types \n \n')
        f.write('{0:.8f} {1:.8f} xlo xhi \n'.format(xlo, xhi))
        f.write('{0:.8f} {1:.8f} ylo yhi \n'.format(ylo, yhi))
        f.write('{0:.8f} {1:.8f} zlo zhi \n\n'.format(zlo, zhi))
        f.write('{0:.8f} {1:.8f} {2:.8f} xy xz yz \n\n'.format(0, 0, yz))            
        f.write('Atoms \n \n')
        np.savetxt(f, FinalMat, fmt='%i %i %.8f %.8f %.8f')
    f.close()

In [198]:
def wrap_at_priodic_boundary_condition(b,c,atoms):
    tol = 0.01
    atoms[:, 2:4] = np.round(atoms[:,2:4],5)
    atoms[np.where(atoms[:,2] >(np.round(b,5)-tol)),2] = atoms[np.where(atoms[:,2] >(np.round(b,5)-tol)),2] - np.round(b,5)
    atoms[np.where(atoms[:,3] >(np.round(c,5)-tol)),3] = atoms[np.where(atoms[:,3] >(np.round(c,5)-tol)),3] - np.round(c,5)
    return atoms

In [326]:
a, b, c, atoms = read_atominfile("atominfile2")

In [327]:
atoms_pbc = wrap_at_priodic_boundary_condition(b,c,atoms)

In [277]:
Write_to_lammps(a,b,c,atoms_pbc,"atominfile")

In [ ]:
gbname = "520"
if float(gbname[1])/float(gbname[0]) < 1/2:
    print("this is under 210")
else:
    print("this is over 210")

ここからが210よりも低角の粒界の凹凸を得るためのコード

In [328]:
gbname = "830"
center = a/2
tol = 1e-5
lattice_constant = 3.61999999500576

In [329]:
atoms_need_modify = atoms_pbc[(atoms_pbc[:,1]>(center + tol- (lattice_constant/2)*np.sin(np.arctan(float(gbname[1])/float(gbname[0]))))) & (atoms_pbc[:,1]<(center - tol + (lattice_constant/2)*np.sin(np.arctan(float(gbname[1])/float(gbname[0])))))]

In [330]:
atoms_need_modify[:,1] = center

In [331]:
reduced_atoms = np.unique(atoms_need_modify,axis=0)

In [332]:
atoms_nocenter = atoms_pbc[(atoms_pbc[:,1]<=(center + tol-(lattice_constant/2)*np.sin(np.arctan(float(gbname[1])/float(gbname[0]))))) | (atoms_pbc[:,1]>=(center - tol + (lattice_constant/2)*np.sin(np.arctan(float(gbname[1])/float(gbname[0])))))]

In [333]:
atoms_final = np.vstack((atoms_nocenter,reduced_atoms))

In [334]:
Write_to_lammps(a,b,c,atoms_final,"atominfile")

ここから210よりも高角の粒界の凹凸を得るためのコード

In [430]:
a, b, c, atoms = read_atominfile("atominfile2")

In [431]:
atoms_pbc = wrap_at_priodic_boundary_condition(b,c,atoms)

In [432]:
gbname = "950"
center = a/2
tol = 1e-3
lattice_constant = 3.61999999500576

In [433]:
atoms_need_modify = atoms_pbc[(atoms_pbc[:,1]>(center + tol- (lattice_constant/2*np.sqrt(2))*abs(np.sin(np.arctan(float(gbname[0])/float(gbname[1]))-(np.pi/4))))) & (atoms_pbc[:,1]<(center - tol + (lattice_constant/2*np.sqrt(2))*abs(np.sin(np.arctan(float(gbname[0])/float(gbname[1]))-(np.pi/4)))))]

In [434]:
atoms_nocenter = atoms_pbc[(atoms_pbc[:,1]<=(center + tol- (lattice_constant/2*np.sqrt(2))*abs(np.sin(np.arctan(float(gbname[0])/float(gbname[1]))-(np.pi/4))))) | (atoms_pbc[:,1]>=(center - tol + (lattice_constant/2*np.sqrt(2))*abs(np.sin(np.arctan(float(gbname[0])/float(gbname[1]))-(np.pi/4)))))]

In [435]:
atoms_need_modify[:,1] = center

In [436]:
reduced_atoms = np.unique(atoms_need_modify,axis=0)

In [437]:
atoms_final = np.vstack((atoms_nocenter,reduced_atoms))

In [438]:
Write_to_lammps(a,b,c,atoms_final,"atominfile")